## Experiment with using GPT and embeddings to detect phenotypes

In [ ]:
import pandas as pd
import json

from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from typing import Tuple, List
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import DataFrameLoader
from dotenv import load_dotenv

load_dotenv()

In [ ]:
with open("../data/raw/hp.json", "r") as f:
    tmp = json.load(f)

hpo_codes = [
    x for x in tmp["graphs"][0]["nodes"] if "type" in x.keys() and x["type"] == "CLASS"
]
hpo_codes

In [ ]:
hpo_codes_clean = [
    {
        "id": x.get("id"),
        "label": x.get("lbl"),
        "definition": x.get("meta", {"definition": {"val": ""}})
        .get("definition", {"val": ""})
        .get("val"),
    }
    for x in hpo_codes
]

hpo_total_df = pd.DataFrame(hpo_codes_clean)
hpo_total_df["content"] = hpo_total_df["label"] + ": " + hpo_total_df["definition"]
hpo_total_df

In [ ]:
hpo_cdb = pd.read_csv("../data/total_cdb.csv").drop(
    columns=["name_status", "ontologies"]
)
# hpo_cdb = hpo_cdb.head(100)
hpo_cdb = hpo_cdb.groupby("cui").agg(lambda x: ", ".join(x)).reset_index()
hpo_cdb

In [ ]:
loader = DataFrameLoader(hpo_cdb, page_content_column="name")
docs = loader.load()
docs

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="xlm-roberta-base")
db = FAISS.from_documents(docs, embeddings)

In [ ]:
query = "patient heeft last van een afwijking aan de blaas"
docs = db.similarity_search(query)
docs

In [ ]:
db.save_local("faiss_index_dutch_hpo")

In [ ]:
from langchain.llms import AzureOpenAI
import os


class HPOTableEntry(BaseModel):
    term_in_text: str = Field(
        description="De term in de tekst die het fenotype beschrijft, bijvoorbeeld: neuropathische blaas"
    )
    description_fenotype: str = Field(
        description="Een beschrijving van het gevonden fenotype, bijvoorbeeld: Blaas die niet goed werkt door beschadiging van een gedeelte van het zenuwstelsel."
    )
    present_in_text: bool = Field(
        description="Is het fenotype aanwezig of niet, bijvoorbeeld: aanwezig of afwezig"
    )
    family_relation: str = Field(
        description="Bij wie is het fenotype geconstateerd, bijvoorbeeld: patiënt, vader of moeder"
    )


class HPOTable(BaseModel):
    entries: List[HPOTableEntry] = Field(
        description="Een fenotype gevonden in de tekst"
    )


# dossier_query = "Patiënt is langer dan normaal en bij nader onderzoek lijkt er sprake te zijn van een afwijking aan de blaas. De moeder van de patiënt heeft een vergrootte baarmoeder. De patient heeft geen AD overerving"
dossier_query = """Een 47-jarige man met een blanco voorgeschiedenis kwam op de polikliniek Dermatologie vanwege erytheem van het gehele rechter been. Het erytheem was al jaren aanwezig en breidde zich langzaam uit van distaal naar proximaal. De patiënt ervoer geen klachten van het aangedane been. Hij had een kantoorbaan en in zijn vrije tijd maakte hij boswandelingen. Hij gebruikte geen insectenwerende middelen en hij kon zich geen tekenbeet herinneren.

Bij lichamelijk onderzoek was een erytheem van het rechter been zichtbaar, zij het erg subtiel. De kleur verschilde met het linker been en er was sprake van een lichte atrofie van de huid rond het enkelgewricht (figuur). Duplexonderzoek liet geen veneuze insufficiëntie zien. Bij serologisch onderzoek was de IgG-antistoftiter tegen Borrelia burgdorferi positief en de IgM-antistoftiter negatief. Hierop stelden wij de diagnose ‘acrodermatitis chronica atroficans’ (ACA).

ACA is een cutane manifestatie van chronische lymeziekte. Unilateraal erytheem of oedeem van een arm of been kan de enige aanwijzing zijn voor ACA. Het wordt vaak ten onrechte aangezien voor veneuze insufficiëntie of lymfoedeem. Bij ACA breidt het erytheem zich uit en gaat het oedeem op den duur over in atrofie. Deze casus laat zien dat het klinische beeld erg subtiel kan zijn.

De patiënt werd volgens het protocol voor chronische lymeziekte behandeld met doxycycline 100 mg 2 dd gedurende 30 dagen. Bij de controle na zes weken was het erytheem aanzienlijk afgenomen qua omvang en kleur."""
# template = r"Extract medical phenotypes from the following patient file. The patient file is in Dutch. Answer in Dutch.\n{format_instructions}\nPatient file: {query}"
template = r"""Haal medische fenotypes uit het volgende patiëntdossier.
    Bijvoorbeeld: Patiënt heeft last van hoofdpijn en prostatitis, maar niet van buikpijn. Zijn moeder heeft een afwijking aan de lichaamslengte
    Dan is het antwoord:
    hoofdpijn, pijn aan het hoofd, aanwezig, patiënt
    prostatitis, een ontsteking aan de prostaat, aanwezig, patiënt
    buikpijn, pijn aan de buik, afwezig, patiënt
    afwijking van lichaamslengte, afwijkende lichaamslengte vergeleken met de norm, aanwezig, moeder

    {format_instructions}

    Patiëntdossier: {query}"""

parser = PydanticOutputParser(pydantic_object=HPOTable)

model = AzureOpenAI(
    temperature=0, deployment_name=os.getenv("DEPLOYMENT_NAME"), engine="model-gpt35"
)

prompt = PromptTemplate(
    template=template,
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

_input = prompt.format_prompt(query=dossier_query)

output = model(_input.to_string())

In [ ]:
parser.get_format_instructions()

In [ ]:
output

In [ ]:
chat_output = parser.parse(output).dict()
chat_output

In [ ]:
embeddings = OpenAIEmbeddings()

db = FAISS.load_local("faiss_index_dutch_hpo", embeddings)
tmp = db.similarity_search_with_relevance_scores(
    ", ".join(
        [
            chat_output["entries"][0]["term_in_text"],
            chat_output["entries"][0]["description_fenotype"],
        ]
    ),
    k=1,
)

In [ ]:
db.similarity_search_with_relevance_scores(
    ", ".join(
        [
            chat_output["entries"][1]["term_in_text"],
            #        chat_output["entries"][1]["description_fenotype"],
        ]
    )
)

In [ ]:
db.similarity_search_with_relevance_scores(
    ", ".join(
        [
            chat_output["entries"][2]["term_in_text"],
            chat_output["entries"][2]["description_fenotype"],
        ]
    )
)

In [ ]:
db.similarity_search_with_relevance_scores(
    ", ".join(
        [
            chat_output["entries"][3]["term_in_text"],
            chat_output["entries"][3]["description_fenotype"],
        ]
    )
)

In [ ]:
db.similarity_search_with_relevance_scores(
    ", ".join(
        [
            chat_output["entries"][4]["term_in_text"],
            chat_output["entries"][4]["description_fenotype"],
        ]
    )
)

In [ ]:
db.similarity_search_with_relevance_scores(
    ", ".join(
        [
            chat_output["entries"][5]["term_in_text"],
            chat_output["entries"][5]["description_fenotype"],
        ]
    )
)

In [ ]:
db.similarity_search_with_relevance_scores(
    "afwijkende lichaamslengte, gegeven de leeftijd wordt een langere lengte verwacht"
)